### Libraries & Dependencies

In [1]:
from pathlib import Path
import json, cv2, shutil, os, random, yaml
from roboflow import Roboflow
from ultralytics import YOLO

### Setup & Folder Structure

In [2]:
base = Path("/content/drive/MyDrive/submission/Part_1_Glove_Detection")
(base/"output").mkdir(parents=True, exist_ok=True)
(base/"logs").mkdir(parents=True, exist_ok=True)

print("Project root:", base)

Project root: /content/drive/MyDrive/submission/Part_1_Glove_Detection


### Importing Dataset

In [3]:
rf = Roboflow(api_key="B5WHJDUX3bA19s3taspE")
project = rf.workspace("glove-detection-3vldq").project("glove-hand-and-bare-hand-zwvif")
version = project.version(1)
dataset = version.download("yolov5")

loading Roboflow workspace...
loading Roboflow project...


In [4]:
print("dataset.location =", dataset.location)
!ls -lah {dataset.location}

dataset.location = /content/Glove-Hand-and-Bare-Hand-1
total 32K
drwxr-xr-x 5 root root 4.0K Nov  1 23:26 .
drwxr-xr-x 1 root root 4.0K Nov  1 23:44 ..
-rw-r--r-- 1 root root  318 Nov  1 23:26 data.yaml
-rw-r--r-- 1 root root  158 Nov  1 23:25 README.dataset.txt
-rw-r--r-- 1 root root 1.1K Nov  1 23:25 README.roboflow.txt
drwxr-xr-x 4 root root 4.0K Nov  1 23:25 test
drwxr-xr-x 4 root root 4.0K Nov  1 23:26 train
drwxr-xr-x 4 root root 4.0K Nov  1 23:26 valid


### YOLOv8 Model

In [5]:
model = YOLO("yolov8n.pt")

model.train(
    data=f"{dataset.location}/data.yaml",
    epochs=30,
    imgsz=640,
    batch=-1,
    name="glove_vs_bare_yolov8n",
    project=str(base / "runs"),
    exist_ok=True
)

Ultralytics 8.3.223 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Glove-Hand-and-Bare-Hand-1/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=glove_vs_bare_yolov8n, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, per

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d1ed41583b0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

### Verifying and Loading Best Weights

In [6]:
!ls /content/runs/detect/glove_vs_bare_yolov8n/weights

best = "/content/drive/MyDrive/submission/Part_1_Glove_Detection/runs/glove_vs_bare_yolov8n/weights/best.pt"

ls: cannot access '/content/runs/detect/glove_vs_bare_yolov8n/weights': No such file or directory


### Inference + Save Annotated Images and JSON Logs

In [7]:
def run_inference_spec(weights, source, out_dir, logs_dir, conf=0.25, imgsz=640):
    """
    - source: folder of .jpg/.jpeg/.png images
    - saves annotated images to out_dir/detect/pred/
    - saves per-image JSONs to logs_dir, with labels gloved_hand / bare_hand
    """
    out_dir  = Path(out_dir);  out_dir.mkdir(parents=True, exist_ok=True)
    logs_dir = Path(logs_dir); logs_dir.mkdir(parents=True, exist_ok=True)

    model = YOLO(weights)
    results = model.predict(
        source=source,
        conf=conf,
        imgsz=imgsz,
        save=True,                 # annotated images
        project=str(out_dir),      # -> out_dir/detect/pred/
        name="pred",
        exist_ok=True,
        verbose=False
    )
    save_dir = Path(results[0].save_dir)

    # Maping dataset names to the required names
    label_map = {"glove_hand": "gloved_hand", "bare_hand": "bare_hand"}

    for r in results:
        dets = []
        if r.boxes is not None and len(r.boxes):
            for xyxy, sc, cl in zip(r.boxes.xyxy.cpu().numpy(),
                                    r.boxes.conf.cpu().numpy(),
                                    r.boxes.cls.cpu().numpy().astype(int)):
                x1, y1, x2, y2 = [int(v) for v in xyxy]
                raw_label = r.names[cl]
                label = label_map.get(raw_label, raw_label)  # enforce spec
                dets.append({"label": label,
                             "confidence": float(sc),
                             "bbox": [x1, y1, x2, y2]})

        json_path = logs_dir / f"{Path(r.path).stem}.json"
        json_path.write_text(json.dumps({"filename": Path(r.path).name,
                                         "detections": dets}, indent=2))

    print("Annotated images:", save_dir)
    print("JSON logs:", logs_dir)

### Run Inference on 5 Validation Images

In [8]:
from pathlib import Path
import shutil

# Defining test source (dataset test images) and destination folder
test_src = Path(dataset.location) / "test" / "images"
test_dst = Path("/content/images_to_test")

# Clean & recreate destination folder
if test_dst.exists():
    shutil.rmtree(test_dst)
test_dst.mkdir(parents=True, exist_ok=True)

# Copy ALL supported image types (.jpg, .jpeg, .png)
for ext in ("*.jpg", "*.jpeg", "*.png"):
    for p in sorted(test_src.glob(ext)):
        shutil.copy(p, test_dst / p.name)

# Runing inference on all test images and saving annotated outputs + JSON logs
run_inference_spec(
    weights=best,
    source=str(test_dst),
    out_dir=str(base / "output"),
    logs_dir=str(base / "logs"),
    conf=0.25,
    imgsz=640
)

Results saved to /content/drive/MyDrive/submission/Part_1_Glove_Detection/output/pred
Annotated images: /content/drive/MyDrive/submission/Part_1_Glove_Detection/output/pred
JSON logs: /content/drive/MyDrive/submission/Part_1_Glove_Detection/logs
